# Neural Nets from Scratch in Julia

## Lesson 5: Backpropagation for addition (part 2)

* In this video we'll finish up the backward pass of the addition operation
* [Documentation site here](https://mikesaint-antoine.github.io/SimpleGrad.jl)
* [Github repo here](https://github.com/mikesaint-antoine/SimpleGrad.jl)

In [1]:
## code so far

mutable struct Value{opType} <: Number
    data::Float64
    grad::Float64
    op::opType
end


struct Operation{FuncType, ArgTypes}
    op::FuncType
    args::ArgTypes
end

# constructor -- Value(data, grad, op)
Value(x::Number) = Value(Float64(x), 0.0, nothing);


import Base.show
function show(io::IO, value::Value)
    print(io, "Value(",value.data,")")
end


import Base.==
function ==(a::Value, b::Value)
     return a===b
end


import Base.+
function +(a::Value, b::Value)

    out = a.data + b.data    
    result = Value(out, 0.0, Operation(+, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end



backprop!(val::Value{Nothing}) = nothing


function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(+), ArgTypes}
    
    # val = a + b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.grad
    val.op.args[2].grad += val.grad
    
end



backprop! (generic function with 2 methods)

In [2]:
function backward(a::Value)
    
    
    function build_topo(v::Value, visited=Value[], topo=Value[])
    
        if !(v in visited)
            
            push!(visited, v)
            
            if v.op != nothing
                for operand in v.op.args
                    
                    if operand isa Value
                        build_topo(operand, visited, topo)
                    end
                end 
            end
            
            push!(topo, v) 
            
            
        end
        return topo
    end
    
    
    
    topo = build_topo(a)
    
    a.grad = 1
    #da/da = 1
    
    for node in reverse(topo)
        backprop!(node)
    end
    
    # trying to get
    # Value[y, x, b, a, z, w, c]
    # topo = Value[x, y, w, a, b, z, c]
    
    
end

backward (generic function with 1 method)

In [3]:
x = Value(2)
y = Value(3)

a = Value(1)
b = Value(4)

w = x + y
z = a + b

c = w + z

backward(c)


println(w.grad) #dc/dw
println(z.grad) #dc/dz

println(a.grad) #dc/da
println(b.grad) #dc/db
println(x.grad) #dc/dx
println(y.grad) #dc/dy


1.0
1.0
1.0
1.0
1.0
1.0


In [4]:
x = Value(3)
y = Value(4)

z = x + y
w = z + x

#dw/dx = 2

backward(w)

println(x.grad)

2.0


In [ ]:
# backprop!() - internal function, called on each variable to update the gradients of the the operands for that variable

# backward() - user facing function, performs full backward pass


# w = x + y
# z = a + b

# c = w + z

# full backward pass
# backward(c)

# backprop!(c) - updating w.grad and z.grad
# backprop!(w) - updating x.grad and y.grad
# backprop!(z) - updating a.grad and b.grad
# backprop!(a) - does nothing
# backprop!(b) - does nothing
# backprop!(x) - does nothing
# backprop!(y) - does nothing

# chain ruling our way back to x
# dc/dx = dc/dw * dw/x